In [3]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

/Users/dohyunkim/Studio-Code/MySQL/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'/Users/dohyunkim/.wdm/drivers/chromedriver/mac64/120.0.6099.109/chromedriver-mac-arm64/chromedriver'

In [9]:
from selenium import webdriver
browser = webdriver.Chrome()

In [ ]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text

In [ ]:
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [27]:
# 1페이지의 링크 데이터 전부 수집

### 한개의 베스트 셀러 링크 수집 (데이터는 요소)
# browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터 (데이터는 리스트)
# datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
# for i in datas:
#     print(i.get_attribute('href'))

### 10페이지 전체의 링크 데이터 ()
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24

import time

link_list = []
for i in range(1,10):
    print("*" * 10, f"현재 {i} 페이지 수집 중 입니다.", "*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
    
    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(1)

len(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
********** 현재 4 페이지 수집 중 입니다. **********
********** 현재 5 페이지 수집 중 입니다. **********
********** 현재 6 페이지 수집 중 입니다. **********
********** 현재 7 페이지 수집 중 입니다. **********
********** 현재 8 페이지 수집 중 입니다. **********
********** 현재 9 페이지 수집 중 입니다. **********


216

In [51]:
# for link in link_list:
#     # 상세 페이지로 이동
#     browser.get(link)
browser.get(link_list[0])
title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]


IndexError: list index out of range

In [40]:
text = "'| 판매지수 164,610 판매지수란?'"
text.split(" ")[2]

'164,610'

In [ ]:
# 상세 페이지 이동 후 데이터 크롤링
browser.get(link_list[0])
title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]

In [62]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장(csv)
import pymysql #.venv이므로 설치 필요
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'password',
    db = 'yes24',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    
    for link in link_list:
        browser.get(link)
        
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 2024년 01월 17일 -> 2024-01-17
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2024-00-00"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text

        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",",""))

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")
        
        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part))   # 문자열이 포함된 데이터에서 숫자만 추출
            except:
                ranking = 0

            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0

        # ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]

        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()
        
        time.sleep(1)

ValueError: invalid literal for int() with base 10: '지역변경'

In [57]:
title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text
# ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[1][:-1]
# ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]

title, author, publisher, publishing, rating, reviews, sales, price, ranking #, ranking_weeks

('푸바오, 언제나 사랑해',
 '에버랜드 동물원 글그림/강철원 글/류정훈 사진',
 '시공주니어',
 '2024년 01월 25일',
 '10.0',
 '14',
 '164,610',
 '23,000',
 '베스트 국내도서 1위')